In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install tqdm torchsummary

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from tqdm import tqdm
from torchsummary import summary
from google.colab import drive
import zipfile
from torchvision.datasets.utils import download_url
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install wandb -qU
import wandb
wandb.login()
# 29eae2ad29417280bdd72cb7ee86bc339759dd9c

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# # Unzip data
#!unzip /content/drive/MyDrive/generated_images_10Kids_cropped.zip -d my_data

In [3]:
if not os.path.exists('./download'):
  # Add your donwload path here
  dataset_url = 'https://drive.google.com/file/d/14swfL7Hz3FbX_DweHYkA8XWmucc8lf0M/view?usp=sharing'
  download_url(dataset_url,'./download', filename='generated_images_10Kids_cropped.zip')

  with zipfile.ZipFile('./download/generated_images_10Kids_cropped.zip', 'r') as tar:
      tar.extractall(path='.')

  image_folder = './generated_images_10Kids_cropped'

Downloading...
From (original): https://drive.google.com/uc?id=14swfL7Hz3FbX_DweHYkA8XWmucc8lf0M
From (redirected): https://drive.usercontent.google.com/download?id=14swfL7Hz3FbX_DweHYkA8XWmucc8lf0M&confirm=t&uuid=b4f26fa5-02af-41d8-bc3b-2e0a43bdd8d5
To: /content/download/generated_images_10Kids_cropped.zip
100%|██████████| 4.02G/4.02G [00:56<00:00, 70.9MB/s]


In [4]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=0)
        self.bn3 = nn.BatchNorm2d(32)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 12 * 12, 41)  # Updated to 32 * 12 * 12
        self.fc1komma5 = nn.Linear(41,32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)

    def forward_one(self, x):
        x = F.relu(self.conv1(x)) # 8 * 112 * 112
        x = F.max_pool2d(x, 2)  # output size: (8, 56, 56)
        x = F.relu(self.conv2(x)) # 16* 52 * 52
        x = F.max_pool2d(x, 2)  # output size: (16, 26, 26)
        x = F.relu(self.conv3(x)) # 32 * 24 * 24
        x = F.max_pool2d(x, 2)  # output size: (32, 12, 12)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc1komma5(x))
        x = F.relu(self.fc2(x))
        return x

    def forward(self, input1, input2, input3, input4):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        output3 = self.forward_one(input3)
        output4 = self.forward_one(input4)
        return output1, output2, output3, output4

In [5]:
class QuadrupletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(QuadrupletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative1, negative2):
        distance_pos = F.pairwise_distance(anchor, positive)
        distance_neg1 = F.pairwise_distance(anchor, negative1)
        distance_neg2 = F.pairwise_distance(positive, negative2)

        loss = torch.mean(F.relu(distance_pos - distance_neg1 + self.margin)) + \
               torch.mean(F.relu(distance_pos - distance_neg2 + self.margin))
        return loss

In [ ]:
# Define custom FaceDataset
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset

class FaceDataset(Dataset):
    def __init__(self, image_folder, people_dirs, transform=None):
        self.image_folder = image_folder
        self.people_dirs = people_dirs
        self.transform = transform
        self.quadruplets = []
        self._prepare_data()

    def _prepare_data(self):
        for person_dir in self.people_dirs:
            person_path = os.path.join(self.image_folder, person_dir)
            images = os.listdir(person_path)
            for i in range(len(images)):
                for j in range(i + 1, len(images)):
                    # Anchor and Positive pair
                    anchor = os.path.join(person_path, images[i])
                    positive = os.path.join(person_path, images[j])

                    # Negative samples
                    neg_person1 = person_dir
                    while neg_person1 == person_dir:
                        neg_person1 = random.choice(self.people_dirs)
                    neg_images1 = os.listdir(os.path.join(self.image_folder, neg_person1))
                    negative1 = os.path.join(self.image_folder, neg_person1, random.choice(neg_images1))

                    neg_person2 = person_dir
                    while neg_person2 == person_dir or neg_person2 == neg_person1:
                        neg_person2 = random.choice(self.people_dirs)
                    neg_images2 = os.listdir(os.path.join(self.image_folder, neg_person2))
                    negative2 = os.path.join(self.image_folder, neg_person2, random.choice(neg_images2))

                    self.quadruplets.append((anchor, positive, negative1, negative2))

    def __len__(self):
        return len(self.quadruplets)

    def __getitem__(self, idx):
        anchor_path, positive_path, negative1_path, negative2_path = self.quadruplets[idx]
        anchor = Image.open(anchor_path).convert('L')
        positive = Image.open(positive_path).convert('L')
        negative1 = Image.open(negative1_path).convert('L')
        negative2 = Image.open(negative2_path).convert('L')

        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative1 = self.transform(negative1)
            negative2 = self.transform(negative2)

        return anchor, positive, negative1, negative2


# Function to split dataset
def split_dataset(image_folder, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    people_dirs = os.listdir(image_folder)
    random.shuffle(people_dirs)

    train_end = int(train_ratio * len(people_dirs))
    val_end = train_end + int(val_ratio * len(people_dirs))

    train_dirs = people_dirs[:train_end]
    val_dirs = people_dirs[train_end:val_end]
    test_dirs = people_dirs[val_end:]

    return train_dirs, val_dirs, test_dirs

# Initialize wandb
wandb.init(project='face-recognition-project')

# Hyperparameters and setup
batch_size = 512
learning_rate = 0.02
epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wandb.config.update({
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "device": str(device)
})

print(f'Batch size: {batch_size}')
print(f'LR: {learning_rate}')
print(f'Epochs: {epochs}')
print(f'Device: {device}')

# Normal transform
transform_normal = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor()
])


# Data augmentation and normalization
transform_data_augmentation = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor()
])

# Load dataset
#image_folder = 'generated_images_10Kids_cropped'  # Update with the path to your dataset
train_dirs, val_dirs, test_dirs = split_dataset(image_folder)

#train_dataset_data_augmentation = FaceDataset(image_folder, train_dirs, transform=transform_data_augmentation)
#train_dataset_normal = FaceDataset(image_folder, train_dirs, transform=transform_normal)

# train_dataset = ConcatDataset([train_dataset_normal, train_dataset_data_augmentation])
train_dataset = FaceDataset(image_folder, train_dirs, transform=transform_data_augmentation)
val_dataset = FaceDataset(image_folder, val_dirs, transform=transform_normal)
test_dataset = FaceDataset(image_folder, test_dirs, transform=transform_normal)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Model, loss, and optimizer
model = SiameseNetwork().to(device)
summary(model, [(1, 112, 112), (1, 112, 112),(1, 112, 112),(1, 112, 112)])
#criterion = nn.BCEWithLogitsLoss()
criterion = QuadrupletLoss(margin=1.0)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
scaler = torch.cuda.amp.GradScaler()

# Training script with validation
import torch
from tqdm import tqdm

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, scaler, epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
            for i, (img1, img2, img3, img4) in enumerate(train_loader):
                img1, img2, img3, img4 = img1.to(device), img2.to(device), img3.to(device), img4.to(device)

                optimizer.zero_grad()

                with torch.cuda.amp.autocast(enabled=scaler is not None):
                    # Pass all four inputs to the model
                    output1, output2, output3, output4 = model(img1, img2, img3, img4)

                    # Calculate loss
                    loss = criterion(output1, output2, output3, output4)

                if scaler is not None:
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()
                pbar.set_postfix(loss=running_loss/(i+1))
                pbar.update(1)

        scheduler.step()

        # Validation
        val_loss, val_accuracy = evaluate(model, val_loader, criterion)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss / len(train_loader)}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": running_loss / len(train_loader),
            "val_loss": val_loss,
            "val_accuracy": val_accuracy
        })

        # Save the model
        torch.save(model.state_dict(), f'./network_epoch{epoch}.pth')

# Evaluation function
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for img1, img2, img3, img4 in data_loader:
            img1, img2, img3, img4 = img1.to(device), img2.to(device), img3.to(device), img4.to(device)
            with torch.cuda.amp.autocast():
                output1, output2, output3, output4 = model(img1, img2, img3, img4)
                loss = criterion(output1, output2, output3, output4)
            running_loss += loss.item()

            # Calculate accuracy (you can adjust this based on your task)
            distance_pos = F.pairwise_distance(output1, output2)
            distance_neg = F.pairwise_distance(output3, output4)
            predicted = (distance_pos < distance_neg).float()  # Adjust as per your task
            # Assuming label is 1 for positive pair and 0 for negative pair
            correct += (predicted == 1).sum().item()
            total += img1.size(0)

    accuracy = correct / total
    return running_loss / len(data_loader), accuracy


# Train the model
#train(model, train_loader, val_loader, criterion, optimizer, epochs=epochs)
train(model, train_loader, val_loader, criterion, optimizer, scheduler, scaler, epochs=epochs)

# Evaluate on test set
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Log final test metrics to wandb
wandb.log({
    "test_loss": test_loss,
    "test_accuracy": test_accuracy
})

# Finish wandb run
wandb.finish()

wandb: Currently logged in as: chrisknaden (da2cs). Use `wandb login --relogin` to force relogin


Batch size: 512
LR: 0.02
Epochs: 5
Device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 112, 112]              80
            Conv2d-2           [-1, 16, 52, 52]           3,216
            Conv2d-3           [-1, 32, 24, 24]           4,640
            Linear-4                   [-1, 41]         188,969
            Linear-5                   [-1, 32]           1,344
            Linear-6                   [-1, 16]             528
            Conv2d-7          [-1, 8, 112, 112]              80
            Conv2d-8           [-1, 16, 52, 52]           3,216
            Conv2d-9           [-1, 32, 24, 24]           4,640
           Linear-10                   [-1, 41]         188,969
           Linear-11                   [-1, 32]           1,344
           Linear-12                   [-1, 16]             528
           Conv2d-13          [-1, 8, 112, 112]        

Epoch 1/5: 100%|██████████| 2598/2598 [31:20<00:00,  1.38batch/s, loss=0.465]


Epoch 1/5, Train Loss: 0.4649793513698519, Val Loss: 0.3347408224024293, Val Accuracy: 0.9268912280701754


Epoch 2/5:   2%|▏         | 48/2598 [00:35<26:05,  1.63batch/s, loss=0.336]